In [27]:
import sys

sys.path.append("../../src")


In [28]:
import time

import joblib
import pandas as pd

# from analysis import (
# plot_confusion_matrix,
# plot_prc_auc_curve,
# plot_roc_auc_curve,
# )
from fitpredictgo import FitPredictGo, Results


In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier


## Загрузка данных

In [30]:
X_train = pd.read_csv("../../data/baseline/X_train.csv")
X_test = pd.read_csv("../../data/baseline/X_test.csv")
y_train = pd.read_csv("../../data/baseline/y_train.csv")
y_test = pd.read_csv("../../data/baseline/y_test.csv")


## Описание исследования

В данном исследовании (эксперименте) предлагается применить алгоритмы, основанные на решающих деревьях, а именно:
1. Классический DesicionTreeClassifier().
2. Ансамблевый RandomForestClassifier().

Планируется осуществить обучение с подбором гиперпараметров.

## Подготовка данных 

Определим признаки для обучения и целевой признак

In [31]:
original_cat_features = []

original_num_features = [
    "age",
    "systolic_bp",
    "diastolic_bp",
    "glucose_level",
    "body_temperature",
    "heart_rate",
]

new_cat_features = ["age_group", "pressure_group"]

new_num_features = []

target = "risk_level"

cat_features = original_cat_features + new_cat_features
num_features = original_num_features + new_num_features


Установим дополнительную константу для указания количества блоков для кросс-валидации.

In [32]:
cv_folds = 5

In [33]:
scoring = "f1_weighted"

In [34]:
RANDOM_STATE = 1206

Определим перечень моделей для обучения. В словарь в будущем внесем результаты обучения моделей.

In [35]:
names = {
    "DecisionTreeClassifier()": None,
    "RandomForestClassifier()": None,
}


Построим пайп-лайны предобработки данных

In [36]:
preprocessor = ColumnTransformer(
    [
        (
            "ord",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
            [0, 1],
        ),
        # ("num", StandardScaler(), [2, 3, 4, 5, 6, 7]),
    ],
    remainder="passthrough",
)


In [37]:
preprocessors = {
    "DecisionTreeClassifier()": [preprocessor],
    "RandomForestClassifier()": [preprocessor],
}


Определим инициализационные модели для кросс-валидации

In [38]:
models = {
    "DecisionTreeClassifier()": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "RandomForestClassifier()": RandomForestClassifier(random_state=RANDOM_STATE),
}


Определим сетку подбора гиперпараметров

In [39]:
param_grid_dtc = [
    {
        "decisiontreeclassifier__max_depth": range(4, 12),
        "decisiontreeclassifier__max_features": range(25, 30),
        "decisiontreeclassifier__class_weight": ["balanced", None],
    }
]


param_grid_rfc = [
    {
        "randomforestclassifier__n_estimators": [50, 100, 150],
        "randomforestclassifier__max_depth": range(5, 9),
        "randomforestclassifier__max_features": range(19, 23),
        "randomforestclassifier__class_weight": ["balanced", None],
    }
]


In [40]:
param_grids = {
    "DecisionTreeClassifier()": param_grid_dtc,
    "RandomForestClassifier()": param_grid_rfc,
}


Выполним обучение. Для каждой модели отображается пайп-лайн для контроля.

In [41]:
%%time

res = Results()
for name in names.keys():
    print(f"{name}, CV")
    model = FitPredictGo(name, scoring)
    start = time.time()
    result = model.fit_cv(
        preprocessors[name],
        models[name],
        X_train,
        y_train["risk_level"],
        cv_folds,
        True,
    )
    time_cv = round(time.time() - start, 3)
    res.update(result)
    print(f"{name}, Randomized Search")
    start = time.time()
    result = model.fit_search(
        preprocessors[name],
        models[name],
        param_grids[name],
        "Randomized",
        X_train,
        y_train["risk_level"],
        cv_folds,
        True,
    )
    time_rs = round(time.time() - start, 3)
    res.update(result)
    names[name] = model
    print(
        f"--> Done. CV Time = {time_cv} секунд,"
        f" RandomizedSearch Time = {time_rs} секунд"
        f"\n\n"
    )
print("All Done")

sorted_results = res.df.sort_values(
    by="Среднее значение метрики при CV", ascending=False
)
sorted_results


DecisionTreeClassifier(), CV


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0, 1])])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=1206))])

DecisionTreeClassifier(), Randomized Search


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0, 1])])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=1206))])

--> Done. CV Time = 0.09 секунд, RandomizedSearch Time = 0.818 секунд


RandomForestClassifier(), CV


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0, 1])])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1206))])

RandomForestClassifier(), Randomized Search


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0, 1])])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1206))])

--> Done. CV Time = 0.97 секунд, RandomizedSearch Time = 8.376 секунд


All Done
CPU times: total: 10.1 s
Wall time: 10.3 s


,Наименование модели,"Время обучения, сек.","Время предсказания, сек.",Среднее значение метрики при CV,СКО метрики при CV
3,Randomized enhanced RandomForestClassifier(),0.150,0.008,0.749,0.031
1,Randomized enhanced DecisionTreeClassifier(),0.007,0.005,0.732,0.042
2,RandomForestClassifier(),0.146,0.009,0.732,0.031
0,DecisionTreeClassifier(),0.007,0.005,0.705,0.032


In [42]:
best_model_1 = names["RandomForestClassifier()"].get_best_estimator("Randomized")

In [43]:
print("Лучшая модель и её параметры:\n\n", best_model_1)

Лучшая модель и её параметры:

 Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0, 1])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=6, max_features=20,
                                        random_state=1206))])


Пройдем Grid поиском в окрестностях параметров, найденных RandomizedSearchCV.

In [44]:
names = {
    "RandomForestClassifier()": None,
}


In [45]:
preprocessors = {
    "RandomForestClassifier()": [preprocessor],
}


In [46]:
models = {
    "RandomForestClassifier()": RandomForestClassifier(
        max_depth=6, max_features=20, random_state=RANDOM_STATE
    ),
}


In [47]:
param_grid_rfc = [
    {
        "randomforestclassifier__max_depth": range(5, 9),
        "randomforestclassifier__max_features": range(18, 25),
    }
]


In [48]:
param_grids = {
    "RandomForestClassifier()": param_grid_rfc,
}


In [49]:
%%time

res = Results()
for name in names.keys():
    model = FitPredictGo(name, scoring)
    print(f"{name}, Grid Search")
    start = time.time()
    result = model.fit_search(
        preprocessors[name],
        models[name],
        param_grids[name],
        "Grid",
        X_train,
        y_train["risk_level"],
        cv_folds,
        True,
    )
    time_rs = round(time.time() - start, 3)
    res.update(result)
    names[name] = model
    print(f"--> Done. GridSearch Time = {time_rs} секунд" f"\n\n")
print("All Done")

sorted_results = res.df.sort_values(
    by="Среднее значение метрики при CV", ascending=False
)
sorted_results


RandomForestClassifier(), Grid Search


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0, 1])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=6, max_features=20,
                                        random_state=1206))])

--> Done. GridSearch Time = 22.515 секунд


All Done
CPU times: total: 22.3 s
Wall time: 22.5 s


,Наименование модели,"Время обучения, сек.","Время предсказания, сек.",Среднее значение метрики при CV,СКО метрики при CV
0,Grid enhanced RandomForestClassifier(),0.145,0.008,0.749,0.031


In [50]:
best_model_2 = names["RandomForestClassifier()"].get_best_estimator("Grid")

In [51]:
print("Лучшая модель и её параметры:\n\n", best_model_2)

Лучшая модель и её параметры:

 Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0, 1])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=6, max_features=18,
                                        random_state=1206))])


## Сохранение модели и данных для дальнейшей работы

Модель, показавшая лучший результат, чем бейзлайн модель после подбора гиперпараметров может быть сохранена, как финальная модель текущего исследования.

In [52]:
joblib.dump(best_model_2, "../../models/exp2_model_749.joblib")

['../../models/exp2_model_749.joblib']

Данные, на которых обучена лучшая модель текущего исследования не изменились. Сохранять их не будем.